# Gaussian Process
reference: http://www.gaussianprocess.org/gpml/chapters/RW.pdf

We have been inferring parameters of function $p(\theta|D)$ rather than the function itself $p(f|D)$. For given input $x_i$ and output $y_i$, we assume there is $y_i = f(x_i )$. The optimal approach is to infer the $p(f|X,y)$ and use this to make predicitons:

$$
\begin{equation}
p(y_*| x_*,X,y) = \int p(y_*|f,x_*)p(f|X,y) df
\end{equation}
$$

Gaussian Process provides an approach to do this. GP defines a prior over functions, which can be converted into a posterior over functions once we have seen some data. Although it's hard to represent a distribution over a function, we can define a distribution over the function's values at a finite, but arbitrary set of points.

GP gives well-calibrated probabilistic outputs, while some kernel methods like LIVM, RVM and SVM, do not.

- ## Regression
Let the prior on the regression function be a GP:

$$
\begin{equation}
f(x) \sim GP(m(x),\kappa(x,x'))
\end{equation}
$$

where,

$$
\begin{align}
m(x) &= E[f(x)] \\
\kappa(x,x') &= E[(f(x)-m(x))(f(x)-m(x'))^T] 
\end{align}
$$

For any finite set of points:

$$
\begin{equation}
p(\mathbf{f|X}) = N(\mathbf{f|\mu,K})
\end{equation}
$$

The mean function is usually set to be 0, as the GP is flexible enough to model the mean arbitrarily well.


### Predictions using noise-free observation

When we assume the observations are noiseless, we expect, for train data $\mathbf{x}$, $f(\mathbf{x})$ has no uncertainty. By definition of the GP, the joint distribution has following form:

$$
\begin{equation}
\begin{pmatrix}f \\ f_* \end{pmatrix}
\sim
N\Big(
\begin{pmatrix}\mu \\ \mu_* \end{pmatrix},
\begin{pmatrix}\mathbf{K} & \mathbf{K_*}\\ \mathbf{K_*^T} & \mathbf{K_{**}} \end{pmatrix}
\Big)
\end{equation}
$$

By the standard fomular for conditioning Gaussian, the posterior has following form:

$$
\begin{align}
p(\mathbf{f_*| X_*,X,f}) & = N(\mathbf{f_*|\mu_*,\Sigma_*}) \\
\mathbf{\mu_*} & = \mathbf{\mu(X_*) + K_*^T K^{-1}(f-\mu(X)) } \\
\mathbf{\Sigma_*} & = \mathbf{K_{**} - K_*^T K^{-1} K_*} 
\end{align}
$$

A squared exponential is a common kernel, aka Gaussian kernel or RBF kernel. In 1d, it is:

$$
\begin{equation}
\kappa(x,x') = \sigma_f^2 exp(-\frac{1}{2l^2}(x-x')^2)
\end{equation}
$$

Hyperparameters $l$ control the horizontal length scle and $\sigma_f^2$ controls the vertical variation.

### Predictions using noisy observations
Assume Gaussian noise, it's easy to get:

$$
\begin{equation}
cov[\mathbf{y|X}] = \mathbf{K} + \sigma_y^2 \mathbf{I}_N \triangleq \mathbf{K}_y
\end{equation}
$$

Therefore, the joint distribution:

$$
\begin{equation}
\begin{pmatrix}\mathbf{y} \\ \mathbf{f_*} \end{pmatrix}
\sim
N\Big(
\mathbf{0},
\begin{pmatrix}\mathbf{K}_y & \mathbf{K_*}\\ \mathbf{K_*^T} & \mathbf{K_{**}} \end{pmatrix}
\Big)
\end{equation}
$$

Posterior:

$$
\begin{align}
p(\mathbf{f_*| X_*,X,y}) & = N(\mathbf{f_*|\mu_*,\Sigma_*}) \\
\mathbf{\mu_*} & = \mathbf{ K_*^T K^{-1}y } \\
\mathbf{\Sigma_*} & = \mathbf{K_{**} - K_*^T K^{-1}_y K_*} 
\end{align}
$$

For single point estimate:

$$
\begin{equation}
\bar{f}_* = \mathbf{k_* K_y^{-1} y} = \sum_{i=1}^N \alpha_i \kappa(\mathbf{x}_i, \mathbf{x}_*)
\end{equation}
$$

Where $\alpha = \mathbf{K}_y^{-1}\mathbf{y}$. The SE kernel to multi-dimension:

$$
\begin{equation}
\kappa_y(\mathbf{x}_p,\mathbf{x}_q) = \sigma_f^2 exp(-\frac{1}{2}(\mathbf{x}_p - \mathbf{x}_q)^T \mathbf{M} (\mathbf{x}_p - \mathbf{x}_q)) + \sigma_y^2 \delta_{pq}
\end{equation}
$$

$\mathbf{M}$ could be defined in many ways, the simplest one is isotropic matrix.

### kernel parameters estimation
Exhaustive grid search is ok but slow. We could also use maximum marginal likelihood estimation:

$$
\begin{equation}
p(\mathbf{y|X}) = \int p(\mathbf{y|f,X}) p(\mathbf{f|X}) d\mathbf{f}
\end{equation}
$$

Everything is Gaussian, therefore we can derive the grad easily:

$$
\begin{equation}
\frac{\partial}{\partial \theta_j}log p(\mathbf{y|X}) = \frac{1}{2} tr\Big((\alpha \alpha^T - \mathbf{K}_y^{-1})\frac{\partial \mathbf{K}_y}{\partial \theta_j}  \Big)
\end{equation}
$$

$\frac{\partial \mathbf{K}_y}{\partial \theta_j}$ depends on the form of kernel. As the variance should be non-negative, we usually take $\theta = log(\sigma_y^2)$